In [4]:
import sqlite3
import pandas as pd
import numpy as np
from flask import Flask
from flask import render_template
import sys
import os

from flask import request, redirect, url_for
from sqlalchemy import func
from flask_sqlalchemy import SQLAlchemy
# from password import password

app = Flask(__name__)

# create_engine('sqlite:///{}'.format('youngtyp.db'), connect_args={'timeout': 15})
DATABASE = 'youngtyp_2.db'
app.config['SQLALCHEMY_DATABASE_URI'] = f'sqlite:///{DATABASE}'
db = SQLAlchemy(app)
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = True
# keyword = password

conn = sqlite3.connect(DATABASE)
cur = conn.cursor()

#создать таблицы в базе данных

cur.execute("""
CREATE TABLE IF NOT EXISTS searches
(id INTEGER PRIMARY KEY AUTOINCREMENT, text text) 
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS texts
(id_text int, text text)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS meta
(id_text int, author text, topic text, year int, affilation text, contacts text, abbr text) 
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS sents 
(id_text int, id_sent int, sent text) 
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS words
(id_text int, id_sent int, id_word int, word text, lemma text, pos text)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS links
(id_text int, id_link int, link text) 
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS glosses
(id_text int, id_gloss int, raw text, gloss text, translation text) 
""")

# conn.commit()
# conn.close()

path = ''

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/no_result')
def no_result():
    return render_template('no_result.html')

@app.route('/credits')
def credits():
    return render_template('credits.html')
    
class Search(db.Model):
    __tablename__ = 'searches'
    id = db.Column(db.Integer, primary_key=True)
    text = db.Column(db.Text)
    
class Words(db.Model):
    __tablename__ = 'words'
    id = db.Column(db.Integer, primary_key=True)
    id_text = db.Column(db.Integer)
    id_sent = db.Column(db.Integer)
    id_word = db.Column(db.Integer)
    word = db.Column(db.Text)
    lemma = db.Column(db.Text)
    pos = db.Column(db.Text)
        
@app.route('/process', methods=['GET', 'POST'])
def answer_process():
    if request.method == 'POST' or not request.args:
        return redirect(url_for('home'))
    text = request.args.get('search')
    search = Search(
        text=text
    )
    db.session.add(search)
    db.session.commit()
    db.session.refresh(search)

    return redirect(url_for('search_result'))

def process_query(query, limit=10):
    '''
    process CQL query

      args:
        query: str, CQL query
        limit: int, returns first N results

      returns:
        result: list of dict, list of dictionaries containing found results

        keys of each dict:
          pattern: str, found pattern
          id_sent: int, id of the sentence with the pattern
          id_text: int, id of the text with the pattern
          id_words: list, list of word indeces
    '''
    return [{'pattern': 'i sit',
        'id_sent': 0,
        'id_text': 0,
        'id_words': [1, 2]},
           
           {'pattern': 'i sit',
        'id_sent': 0,
        'id_text': 0,
        'id_words': [1, 2]}]

def make_left(sents, id_sent, id_words, length=5):
    left = sents[id_sent][:id_words[0]][::-1]
    id_sent -= 1
    while len(left) < length and id_sent >= 0:
        n_needed = length - len(left)
        if len(sents[id_sent]) < n_needed:
            n_needed = len(sents[id_sent])
        fetch = sents[id_sent].split()[-n_needed::]
        left += reversed(fetch)
        id_sent -= 1
    return ' '.join(left[::-1])

def make_right(sents, id_sent, id_words, length=5):
    right = sents[id_sent][id_words[1]+1:]
    id_sent += 1
    while len(right) < length and id_sent < len(sents):
        n_needed = length - len(right)
        if len(sents[id_sent]) < n_needed:
            n_needed = len(sents[id_sent])
        fetch = sents[id_sent].split()[:n_needed]
        right += fetch
        id_sent += 1
    return ' '.join(right)

@app.route('/search_result')
def search_result():
    text = db.session.query(Search.text).order_by(Search.id.desc()).where(Search.text != None).first()[0]
    results = process_query(text)
    
    outputs = []
    for result in results:
        id_sents = [ele[0] for ele in db.engine.execute(
                f'''
                SELECT DISTINCT id_sent FROM words WHERE id_text={result['id_text']}
                ''')]
        sents = [[ele[0] for ele in db.engine.execute(
                f'''
                SELECT DISTINCT word FROM words WHERE id_text={result['id_text']}
                                            AND id_sent={id_sent}
                ''')] for id_sent in id_sents]
        
        meta = [ele for ele in db.engine.execute(
                f'''
                SELECT DISTINCT author, topic, year, affilation
                FROM meta WHERE id_text={result['id_text']}
                ''')][0]

        output = dict()
        output['right'] = make_right(sents, result['id_sent'], result['id_words'], length=5)
        output['left'] = make_left(sents, result['id_sent'], result['id_words'], length=5)
        output['pattern'] = result['pattern']
        output['author'] = meta[0]
        output['topic'] = meta[1]
        output['year'] = meta[2]
        output['affiliation'] = meta[3]
        outputs.append(output)
    
    if outputs == []:
        return render_template('no_result.html')
    else:
        return render_template('output.html', outputs=outputs)

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/shikunova/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Oct/2022 19:40:51] "GET /search_result HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2022 19:55:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2022 19:55:05] "GET /static/search.gif HTTP/1.1" 200 -


In [45]:
results = [{'pattern': 'i sit',
        'id_sent': 0,
        'id_text': 0,
        'id_words': [1, 2]}]

[ele[0] for ele in db.engine.execute(
            f'''
            SELECT id_sent FROM words WHERE id_text=0
            ''')]

[]

In [86]:
res = {'pattern': 'to shit',
        'id_sent': 0,
        'id_text': 0,
        'id_words': [1, 2]}
sents = ['here i sit', 'broken hearted', 'came to shit but only farted']
text = 'here i sit broken hearted came to shit but only farted'

def make_left(sents, id_sent, id_text, id_words, length=5):
    left = sents[id_sent].split()[:id_words[0]][::-1]
    id_sent -= 1
    while len(left) < length and id_sent >= 0:
        n_needed = length - len(left)
        if len(sents[id_sent]) < n_needed:
            n_needed = len(sents[id_sent])
        fetch = sents[id_sent].split()[-n_needed::]
        left += reversed(fetch)
        id_sent -= 1
    return ' '.join(left[::-1])
    
make_left(sents, 0, 0, [2, 3])

'here i'

In [82]:
def make_right(sents, id_sent, id_text, id_words, length=5):
    right = sents[id_sent].split()[id_words[1]+1:]
    id_sent += 1
    while len(right) < length and id_sent < len(sents):
        n_needed = length - len(right)
        if len(sents[id_sent]) < n_needed:
            n_needed = len(sents[id_sent])
        fetch = sents[id_sent].split()[:n_needed]
        right += fetch
        id_sent += 1
    return ' '.join(right)
    
make_right(sents, 0, 0, [0, 1])

'sit broken hearted came to'

In [125]:
DATABASE = 'youngtyp_2.db'
con = sqlite3.connect(DATABASE)  # подключение
cur = con.cursor()  # курсор

In [121]:
# texts: id_text, text
# meta: id_text, author, topic, year, affilation, contacts, abbr text
# links: id_text, id_link, link
# glosses: id_text, id_gloss, raw, gloss, translation
# sents: id_text, id_sent, sent
# words: id_text, id_sent, id_word, word, lemma, pos

queries = '''INSERT INTO texts (id_text, text)
VALUES (0, 'here i sit broken hearted came to shit but only farted');

INSERT INTO sents (id_text, id_sent, sent)
VALUES (0, 0, 'here i sit');

INSERT INTO sents (id_text, id_sent, sent)
VALUES (0, 1, 'broken hearted');

INSERT INTO sents (id_text, id_sent, sent)
VALUES (0, 2, 'came to shit but only farted');

INSERT INTO meta (id_text, author, topic, year, affilation, contacts, abbr)
VALUES (0, 'sasha', 'test', 2002, 'HSE', '88005553535', 'A=ass P=pee')'''.split(';\n\n')

for query in queries:
    cur.execute(query)
cur.fetchall()

[]

In [126]:
cur.execute('select * from meta')
cur.fetchall()

[(0, 'sasha', 'test', 2002, 'HSE', '88005553535', 'A=ass P=pee')]

In [124]:
con.commit()
con.close()